### TensorBoard in PaperSpace

https://docs.paperspace.com/gradient/notebooks/tensorboard

In [ ]:
NOTEBOOK="nolw4352wz"
print(f"https://tensorboard-{NOTEBOOK}.clg07azjl.paperspacegradient.com")

### Training the model

In [ ]:
import src.lightning as l

import torch
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, Callback

import os

In [ ]:
en, de = l.vocabs()
train, val, test = l.batched_data_loaders((en, de), 2000)

In [ ]:
checkpoint_path = 'checkpoints/iwslt2017'
chkpt_model_path = 'checkpoints/iwslt2017/'

if os.path.exists(chkpt_model_path):
    model = l.LightningSeq2Seq.load_from_checkpoint(chkpt_model_path, src_vocab=len(en), trg_vocab=len(de))
else:
    model = l.LightningSeq2Seq(len(en), len(de), warmup=4000, N=6, factor=1)

In [ ]:
trainer = pl.Trainer(
    default_root_dir=checkpoint_path,
    gpus=1,
    max_epochs=50,
    callbacks=[
        # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
        ModelCheckpoint(
            save_weights_only=False, save_top_k=1,
            mode="min", monitor="val_loss",
            every_n_epochs=1
        ),
        # Log learning rate every epoch
        LearningRateMonitor("step"),
        l.LogDistributions()
    ],
    fast_dev_run=False,
    log_every_n_steps=25,
    accumulate_grad_batches=5
)

In [ ]:
trainer.fit(model, train, val)